In [1]:
import torch

### Tensor haqida tushuncha

In [4]:
x = torch.tensor([1, 2, 3])
y = torch.tensor([4, 5, 6])
z = x * y
print(z)

tensor([ 4, 10, 18])


In [7]:
# reshape the tensor with view
a = torch.tensor([[1, 2, 3], [4, 5, 6]])
print(a.shape)
b = a.view(3, 2)
print(b.shape)

torch.Size([2, 3])
torch.Size([3, 2])


In [10]:
# autogran and backpropagation with pytorch
x = torch.tensor([1.0], requires_grad=True)
y = 2 * x
z = y ** 2
z.backward()
print(x.grad)

tensor([8.])


### Chuqur O'rganish Modeli